# 以相似度为训练目标

## 生成训练数据，暂存于本地

In [0]:
!git clone https://github.com/daibiaoxuwu/05wan3

In [0]:
!git clone https://github.com/daibiaoxuwu/05wan3b

In [0]:
import os
import cv2
import numpy as np
path1 = '05wan3'
path2 = '05wan3b/0'

clocks = dict()
x_train0 = []
y_train0 = []
for filename in os.listdir(path2):
    if 'git' in filename: continue
    #print(path1)
    #ftimestr,number = path1.split('_')
    clocks[int(filename.split('_')[0])]=int(filename.split('_')[1][:-4])
    #clocks[int(ftimestr)]=int(number)
keys = clocks.keys()
for filename in os.listdir(path1):
    if 'git' in filename or 'c.jpg' in filename: continue
    ftime = int(filename.split('_')[0])
    value = max(filter(lambda t:t<ftime,keys))
    x_train0.append(cv2.imread(os.path.join(path1,filename), cv2.IMREAD_GRAYSCALE)/255.0)
    y_train0.append(clocks[value])
    
print(len(x_train0),len(y_train0))

In [0]:
import seaborn as sns
sns.countplot(y_train0)
print(y_train0[:100])

# 转为onehot，分离test和train

# 装载Tensorflow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# 安装 TensorFlow
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

# 输入数据生成器
# 变为5张图片一叠

In [0]:
idxes = [[]]*10
for idx, val in enumerate(y_train0):
  idxes[val].append(idx)

import random

def train_generator():
  for train in range(60000):
      ans = random.randint(0,9)
      idx2 = random.sample(idxes[ans],5)
      data = np.stack([x_train0[ix] for ix in idx2],axis=2)
      ansy = np.zeros(10)
      ansy[ans] = 1
      yield data,ansy


train_dataset = tf.data.Dataset.from_generator(train_generator, (tf.float32, tf.float32), (tf.TensorShape([12,12,5]),tf.TensorShape([10,])))
train_dataset = train_dataset.batch(32)

# 构建模型，优化器和损失函数：

In [0]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5, 5),padding = 'Same',  activation='relu', input_shape=(12, 12, 5)),
    tf.keras.layers.Conv2D(32, (5, 5),padding = 'Same',  activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3, 3),padding = 'Same',  activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3),padding = 'Same',  activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = 'adam' , loss = "categorical_crossentropy", metrics=["accuracy"])


#histor = model.fit_generator(train_generator(),epochs = 50, steps_per_epoch=2000)
model.fit(train_dataset,epochs=50)

# 训练并验证模型：